### 1. 安装aws、rclone 环境

In [ ]:
!pip install --user awscli boto3
!pip install kora
!curl https://rclone.org/install.sh | sudo bash
!sudo bash install.sh
!rclone --version

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.8 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 688.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 

### 2. 配置aws和rclone[tos]

In [ ]:
import os
os.environ['PATH'] = f"~/.local/bin:{os.environ['PATH']}"
tos="aws --endpoint-url http://tos-s3-cn-shanghai.volces.com s3"
# 配置awscli
!aws configure
content="""
[default]
aws_access_key_id = XXX
aws_secret_access_key = XXX
region = cn-shanghai
s3 =
    addressing_style = virtual
    endpoint_url = http://tos-s3-cn-shanghai.ivolces.com
"""

os.makedirs("/root/.aws", exist_ok=True)

with open("/root/.aws/config",'w') as f:
  f.write("[default]")

with open("/root/.aws/credentials","w") as f:
  f.write(content)

# 指定配置内容
config_content = """
[tos]
type = s3
provider = Other
access_key_id = XXX
secret_access_key = XXX
region = cn-shanghai
endpoint = http://tos-s3-cn-shanghai.ivolces.com
force_path_style = false
disable_http2 = true
"""

# 将配置内容写入文件
with open('/root/.config/rclone/rclone.conf', 'w') as f:
    f.write(config_content)



### 3. 测试是否配置成功

In [ ]:
!{tos} ls 


2024-01-27 14:07:34    6026003 abaQDWvhvGw.mp4
2024-01-25 03:59:16        162 test.mp4
2024-01-27 14:05:45    6026003 test1.mp4


### 4. 直接wget，下面实现的是视频下载与上传

In [ ]:
import requests
import boto3
from tqdm import tqdm
import os
import multiprocessing
from multiprocessing import Pool

# 配置
download_list = "/content/drive/MyDrive/howtolink/data/all_train.txt"  # 下载列表文件路径
user_name = "htlog23"  # 用户名
password = "fb93dc3b1950d18"  # 密码
bucket_name = 'kanelin'  # S3 bucket 名称

# 初始化 S3 客户端
s3 = boto3.client('s3', endpoint_url='http://tos-s3-cn-shanghai.volces.com',
                  aws_access_key_id='AKLTMGFkMWY5MDE5ZjVjNGZjYzkwYTk2Yzk5ODY3OWQxYjY',
                  aws_secret_access_key='WkRBME5qSmxaV1U1TW1ZMU5ETTNNbUptWlRCaE9EbGtZbU01WXpobFptVQ==')

# 获取 S3 上已上传的文件列表
def get_uploaded_videos():
    uploaded_videos = []
    paginator = s3.get_paginator('list_objects_v2')
    for page in paginator.paginate(Bucket=bucket_name, Prefix='Howto-Interlink7M_subset_w_all_clips_train/'):
        if 'Contents' in page:
            for obj in page['Contents']:
                uploaded_videos.append(obj['Key'])
    return uploaded_videos

def process_video(video_url):
    try:
        download_and_upload(video_url)
    except:
        pass
    return video_url

# 下载并上传视频的函数
def download_and_upload(video_url):
    video_url ="http://howto100m.inria.fr/dataset/" + video_url
    local_filename = video_url.split('/')[-1]
    local_path = f"/content/{local_filename}"
    s3_key = f'Howto-Interlink7M_subset_w_all_clips_train/{local_filename}'

    # Check if the video was already uploaded
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=s3_key)
    if 'Contents' in response:  # The file is already on S3
        print(f"The file {local_filename} already exists on S3.\n")
        return

    # Download video with streaming
    with requests.get(video_url, auth=(user_name, password), stream=True) as r:
        r.raise_for_status()
        with open(local_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

    if os.path.exists(local_path) and os.path.getsize(local_path) > 5000:
        # 上传到 S3-compatible service
        s3.upload_file(local_path, bucket_name, s3_key)
        print(f"Successfully uploaded {local_filename}\n")

        # Optionally: Remove the local file after successful upload
        os.remove(local_path)
    else:
        # 处理下载失败的情况
        print(f"Failed to download video from {video_url}\n")


# 读取下载列表并处理每个视频
uploaded_videos = get_uploaded_videos()
uploaded_filenames = [video.split('/')[-1] for video in uploaded_videos]
remaining_videos = []

with open(download_list, 'r') as file:
    all_videos = [video_url.strip().split('/')[-1] for video_url in file]

# 将已上传的文件名列表转换为集合
uploaded_filenames_set = set(uploaded_filenames)

# 将所有视频链接的文件名转换为集合
all_videos_set = set(all_videos)

# 使用集合减法操作，得到未上传的视频列表
remaining_videos = list(all_videos_set - uploaded_filenames_set)

print(f"已完成: {len(uploaded_filenames)}\n未完成: {len(remaining_videos)}\n")

# 进程池处理未处理的视频
num_processes = 128  # 设置进程数
with Pool(processes=num_processes) as pool:
    for _ in tqdm(pool.imap_unordered(process_video, remaining_videos), total=len(remaining_videos)):
        pass

print('所有文件上传成功！\n')

### 5. 上传至tos

In [ ]:
!{tos} cp local_path s3://xxxx

fatal error:                                                       
^C
